# D'COVIA

Covid-19 Detection based on Lungs X-Ray Image Using Convolutional Neural Network Algorithm

In [ ]:
from ipywidgets import Layout, Button, Box, FileUpload, Label, Checkbox, IntSlider, IntText, Button
import ipywidgets as widgets
from IPython.display import display, Image
import threading
import time

import tensorflow as tf
#
from keras.preprocessing import image

# uploader to image cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt
 
import os

In [ ]:
h5_path = 'modellpb.h5'
model = tf.keras.models.load_model(h5_path)

In [ ]:
# uploader
annotate_image_uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )


In [ ]:
# uploader container
image_annotate_container = widgets.VBox(
    children = ()
)

In [ ]:
# wrapper
app_2 = widgets.VBox(
    children = (annotate_image_uploader, image_annotate_container,)
)

In [ ]:
# uploader image content and extension
def get_content_and_extension(annotate_image_uploader):    
    for _, value in annotate_image_uploader.value.items():
        return value['content'], value['metadata']['name'].split('.')[-1]

In [ ]:
# image widget from annotate_image_uploader
def image_widget_from_uploader(annotate_image_uploader):
    if annotate_image_uploader.value:
        file_content, file_extension = get_content_and_extension(annotate_image_uploader)
        return widgets.Image(value=file_content, format=file_extension)
    else:
        warnings.warn("NO IMAGE HAS BEEN UPLOADED", category=Warning)
        return widgets.Image()

In [ ]:
# image numpy from uploader
def image_numpy_from_uploader(annotate_image_uploader):
    file_content, extension = get_content_and_extension(annotate_image_uploader)
    arr_from_buf = np.frombuffer(file_content, np.uint8)
    image_numpy = cv2.imdecode(arr_from_buf, cv2.IMREAD_COLOR)
    return image_numpy, extension

In [ ]:
# image widget from image numpy
def image_widget_from_numpy(image_numpy, extension):
    _, encoded_image = cv2.imencode('.'+extension, image_numpy)
    image_bytes = encoded_image.tobytes()
    return widgets.Image(value=image_bytes, format=extension)

In [ ]:
def image_reshape(image):
    image = cv2.resize(image, (256, 256))
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    image = image.reshape(-1, 256, 256, 3)
    return image

In [ ]:
def prepro_greyscale(image_numpy):
    return cv2.cvtColor(image_numpy, cv2.COLOR_BGR2GRAY)

def prepro_resize(image_numpy,resize_w=256,resize_l=256):
    return cv2.resize(image_numpy,(resize_w,resize_l))

def prepro_normalize(image_numpy,resize_w=256,resize_l=256):
    out_arr = np.zeros((resize_w,resize_l))
    return cv2.normalize(image_numpy, out_arr, 0, 255, cv2.NORM_MINMAX)

def prepro_clahe(image_numpy, clipLimit=0.2, tileGridSize=8):
    clahe = cv2.createCLAHE(clipLimit, (tileGridSize, tileGridSize))
    return clahe.apply(image_numpy)

In [ ]:
def predicted_image(img_reshape_2, model):
# model = model(input_shape= (32,32,1))
    result = model.predict(img_reshape_2)
    if result.any() == 0:
        prediction = 'Normal'
    elif result.any() == 1:
        prediction = 'Covid-19'
    else:
        prediction = 'Bacteria'

  # prediction =  print(prediction)
    result = ' ' + str(prediction) + ' '
    return result

In [ ]:
text_result = widgets.Label(value="Result will be shown here!")

In [ ]:
text = widgets.VBox(
    children = ([text_result]), layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    ))

In [ ]:
# uploader container
text_container = widgets.VBox(
    children = (text,)
)

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

In [ ]:
# label
label_current_thread = widgets.Label()
label_thread_list = widgets.Label()

# button 
button_stop = widgets.Button(
    description = 'stop',
    icon = 'fa-stop',
    button_style = 'warning',
    layout = widgets.Layout(width='100px')
)
def button_stop_click(self):
    global thread_status
    thread_status = False
button_stop.on_click(button_stop_click)

button_check_thread = widgets.Button(
    description = 'check thread',
    button_style = '',
    layout = widgets.Layout(width='100px')
)
def button_check_thread_click(self):
    # print if work() still running
    work_count = 0
    name='work'
    for thread_object in threading.enumerate():
        if thread_object.name == name:
            work_count+=1
    if work_count > 0:
        label_thread_list.value=f'{work_count} thread with name {name} still exist. Consider use exit() and restart kernel'
    else:
        label_thread_list.value=f'No thread with name {name} exist'
button_check_thread.on_click(button_check_thread_click)

In [ ]:
# main loop
def work():
    global thread_status
    global model
    global image_numpy
    global img_reshape_2
    
    if thread_status:
        label_current_thread.value = 'Thread On'

    current_state_2 = [annotate_image_uploader.value]
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_state_2 !=[annotate_image_uploader.value]:
            
            # Preprocessing message
            text_result.value = 'Preprocessing...'
            text.children = (text_result,)
            if text_result.value:
                text_container.children = (text,)
            
            # renew state reference
            current_state_2 = [annotate_image_uploader.value]         
            
            image_widget_annotate = image_widget_from_uploader(annotate_image_uploader)
            
            image_numpy, extension = image_numpy_from_uploader(annotate_image_uploader)
            
            # preprocess and predict
            
            image_numpy = prepro_greyscale(image_numpy)

            image_numpy = prepro_resize(image_numpy)

            image_numpy = prepro_normalize(image_numpy)
            
            image_numpy = prepro_clahe(image_numpy)
        
            image_numpy_widget = image_widget_from_numpy(image_numpy, extension)
            
            image_numpy = image_reshape(image_numpy)
            
            image_annotate_container.children = (image_numpy_widget,)
            
            # Predict message message
            text_result.value = 'Predicting...'
            text.children = (text_result,)
            if text_result.value:
                text_container.children = (text,)
            
            text_result.value = predicted_image(image_numpy, model)
            text.children = (text_result,)
            if text_result.value:
                text_container.children = (text,)
                    
        # sleep for threading
        time.sleep(0.05)
        
        if not thread_status:
            label_current_thread.value = 'Thread Off'

In [ ]:
# restart flag
thread_status = False  
thread_status = True

# create thread
thread_work = threading.Thread(target=work)
thread_work.name = 'work'

# start thread
thread_work.start()

In [ ]:
# Box([Label(value='Upload Mammography'), app], layout=form_item_layout)

In [ ]:
Box([Label(value='Upload Lungs X-Ray Image'), app_2], layout=form_item_layout)

In [ ]:
Box([Label(value='Result'), text_container], layout=form_item_layout)